In [18]:
import pandas as pd
from datetime import datetime
from sklearn.cross_validation import train_test_split
import numpy as np
import rpy2.robjects as ro
from sklearn.metrics import mean_squared_error
import math

In [19]:
# Read orders data
order_items = pd.read_csv('order_items.csv')
order_items.head()

,trip_id,item_id,department_name,quantity
0,3119513,368671,Produce,10.0
1,3120462,368671,Produce,10.0
2,3120473,368671,Produce,10.0
3,3121910,368671,Produce,6.0
4,3122332,368671,Produce,10.0


In [20]:
# Generate Varibles from orders data to be used for training model and predictions.
IDV_data = pd.DataFrame()

# Get number of items for a trip_id
IDV_data = order_items[ [ 'item_id', 'trip_id' ]].groupby(['trip_id'], as_index=False ).count()
IDV_data.columns = ['trip_id', 'item_count']
IDV_data.set_index('trip_id')
IDV_data.head()

,trip_id,item_count
0,3119513,33
1,3119516,5
2,3119518,12
3,3119519,26
4,3119520,33


In [21]:
# Get sum of quantity of items for a trip_id
temp_data = order_items[ [ 'quantity', 'trip_id' ]].groupby(['trip_id'], as_index=False ).sum()
temp_data.columns = ['trip_id', 'quantity_sum']
temp_data.set_index('trip_id')
IDV_data = pd.merge( IDV_data, temp_data, left_on='trip_id', right_on='trip_id', how='left')[ 1:10 ]

In [22]:
# Find the most popular departments
popular_depts = order_items[ 'department_name' ].value_counts( )[ 0: 10 ].index.values
# print popular_depts

In [23]:
for dept_name in popular_depts:
    # Filter by dept_name
    order_items_filtered = order_items[ order_items[ 'department_name' ] == dept_name ] 
    # remove spacing and & in dept_name
    dept_name = dept_name.replace( ' ', '_' )
    dept_name = dept_name.replace( '&', 'and' )

    # For each dept in popular depts find the number of items per trip.
    item_count = order_items_filtered[ [ 'item_id', 'trip_id' ]].groupby(['trip_id'], as_index=False ).count() 
    item_count.columns = ['trip_id', dept_name + '_count' ]
    item_count.set_index( 'trip_id' )
    IDV_data = pd.merge( IDV_data, item_count, left_on='trip_id', right_on='trip_id', how='left')
    
    # For each dept in popular depts find the quantity sum per trip.
    item_count = order_items_filtered[ [ 'quantity', 'trip_id' ]].groupby(['trip_id'], as_index=False ).sum() 
    item_count.columns = ['trip_id', dept_name + '_quantity_sum' ]
    item_count.set_index( 'trip_id' )
    IDV_data = pd.merge( IDV_data, item_count, left_on='trip_id', right_on='trip_id', how='left')

In [24]:
IDV_data.head()

,trip_id,item_count,quantity_sum,Popular_count,Popular_quantity_sum,Produce_count,Produce_quantity_sum,Dairy_and_Eggs_count,Dairy_and_Eggs_quantity_sum,Snacks_count,...,Pantry_count,Pantry_quantity_sum,Frozen_count,Frozen_quantity_sum,Deli_count,Deli_quantity_sum,Bakery_count,Bakery_quantity_sum,Meat_and_Seafood_count,Meat_and_Seafood_quantity_sum
0,3119516,5,9.0,1,4.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3119518,12,21.0,3,9.0,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN
2,3119519,26,39.0,4,8.0,3.0,3.0,1.0,1.0,1.0,...,NaN,NaN,2.0,3.0,4.0,7.0,2.0,2.0,1.0,1.0
3,3119520,33,36.0,4,4.0,4.0,4.0,6.0,8.0,2.0,...,2.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0
4,3119703,32,40.0,12,17.0,3.0,4.0,2.0,2.0,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,2.0,2.0,NaN,NaN


In [25]:
# Read the train_trips csv
train_trips = pd.read_csv('train_trips.csv')
train_trips.set_index('trip_id')
train_trips.head()
# To make it clear that store_id is a categorical variable and not a integar append the column with string 'store_id_'
train_trips[ 'store_id' ] = 'store_id_' + train_trips.store_id.map(str)
train_trips[ 'shopper_id' ] = 'shopper_id_' + train_trips.shopper_id.map(str)
train_trips.head()

,trip_id,shopper_id,fulfillment_model,store_id,shopping_started_at,shopping_ended_at
0,3119519,shopper_id_48539,model_1,store_id_6,2015-09-01 07:03:56,2015-09-01 07:30:56
1,3119513,shopper_id_3775,model_1,store_id_1,2015-09-01 07:04:33,2015-09-01 07:40:33
2,3119516,shopper_id_4362,model_1,store_id_1,2015-09-01 07:23:21,2015-09-01 07:41:21
3,3119792,shopper_id_47659,model_1,store_id_1,2015-09-01 07:29:52,2015-09-01 08:55:52
4,3119922,shopper_id_11475,model_1,store_id_1,2015-09-01 07:32:21,2015-09-01 09:01:21


In [26]:
# Convert the columns for string to datetime.
train_trips[ 'shopping_started_at' ] = train_trips[ 'shopping_started_at' ].apply( lambda x: datetime.strptime( x, '%Y-%m-%d %H:%M:%S' ) )
train_trips[ 'shopping_ended_at' ] = train_trips[ 'shopping_ended_at' ].apply( lambda x: datetime.strptime( x, '%Y-%m-%d %H:%M:%S' ) )

In [27]:
train_trips[ "shopping_time" ] = ( train_trips[ 'shopping_ended_at' ] - train_trips[ 'shopping_started_at' ] ).apply( lambda x: x.total_seconds() )
train_trips.head()

,trip_id,shopper_id,fulfillment_model,store_id,shopping_started_at,shopping_ended_at,shopping_time
0,3119519,shopper_id_48539,model_1,store_id_6,2015-09-01 07:03:56,2015-09-01 07:30:56,1620.0
1,3119513,shopper_id_3775,model_1,store_id_1,2015-09-01 07:04:33,2015-09-01 07:40:33,2160.0
2,3119516,shopper_id_4362,model_1,store_id_1,2015-09-01 07:23:21,2015-09-01 07:41:21,1080.0
3,3119792,shopper_id_47659,model_1,store_id_1,2015-09-01 07:29:52,2015-09-01 08:55:52,5160.0
4,3119922,shopper_id_11475,model_1,store_id_1,2015-09-01 07:32:21,2015-09-01 09:01:21,5340.0


In [28]:
# Feature Engineering Pipeline
model_dummies = pd.get_dummies( train_trips[ "fulfillment_model" ] )
train_trips = pd.concat( [ train_trips, model_dummies], axis = 1 )
# drop model_2 for multi collinearity 
train_trips.drop(['fulfillment_model', 'model_2'], inplace=True, axis=1)
train_trips.head()

,trip_id,shopper_id,store_id,shopping_started_at,shopping_ended_at,shopping_time,model_1
0,3119519,shopper_id_48539,store_id_6,2015-09-01 07:03:56,2015-09-01 07:30:56,1620.0,1.0
1,3119513,shopper_id_3775,store_id_1,2015-09-01 07:04:33,2015-09-01 07:40:33,2160.0,1.0
2,3119516,shopper_id_4362,store_id_1,2015-09-01 07:23:21,2015-09-01 07:41:21,1080.0,1.0
3,3119792,shopper_id_47659,store_id_1,2015-09-01 07:29:52,2015-09-01 08:55:52,5160.0,1.0
4,3119922,shopper_id_11475,store_id_1,2015-09-01 07:32:21,2015-09-01 09:01:21,5340.0,1.0


In [29]:
store_dummies = pd.get_dummies( train_trips[ "store_id" ] )
train_trips = pd.concat( [ train_trips, store_dummies ], axis = 1 )
# drop 1 for multi collinearity 
train_trips.drop( ['store_id', 'store_id_1'], inplace=True, axis=1)
train_trips.head()

,trip_id,shopper_id,shopping_started_at,shopping_ended_at,shopping_time,model_1,store_id_105,store_id_115,store_id_123,store_id_126,store_id_148,store_id_29,store_id_3,store_id_31,store_id_5,store_id_54,store_id_6,store_id_78,store_id_90
0,3119519,shopper_id_48539,2015-09-01 07:03:56,2015-09-01 07:30:56,1620.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3119513,shopper_id_3775,2015-09-01 07:04:33,2015-09-01 07:40:33,2160.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3119516,shopper_id_4362,2015-09-01 07:23:21,2015-09-01 07:41:21,1080.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3119792,shopper_id_47659,2015-09-01 07:29:52,2015-09-01 08:55:52,5160.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3119922,shopper_id_11475,2015-09-01 07:32:21,2015-09-01 09:01:21,5340.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
train_trips = pd.merge( train_trips, IDV_data, left_on='trip_id', right_on='trip_id', how='left' )
train_trips.head()

,trip_id,shopper_id,shopping_started_at,shopping_ended_at,shopping_time,model_1,store_id_105,store_id_115,store_id_123,store_id_126,...,Pantry_count_y,Pantry_quantity_sum_y,Frozen_count_y,Frozen_quantity_sum_y,Deli_count_y,Deli_quantity_sum_y,Bakery_count_y,Bakery_quantity_sum_y,Meat_and_Seafood_count_y,Meat_and_Seafood_quantity_sum_y
0,3119519,shopper_id_48539,2015-09-01 07:03:56,2015-09-01 07:30:56,1620.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,2.0,3.0,4.0,7.0,2.0,2.0,1.0,1.0
1,3119513,shopper_id_3775,2015-09-01 07:04:33,2015-09-01 07:40:33,2160.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3119516,shopper_id_4362,2015-09-01 07:23:21,2015-09-01 07:41:21,1080.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3119792,shopper_id_47659,2015-09-01 07:29:52,2015-09-01 08:55:52,5160.0,1.0,0.0,0.0,0.0,0.0,...,5.0,5.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
4,3119922,shopper_id_11475,2015-09-01 07:32:21,2015-09-01 09:01:21,5340.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,3.0,8.0,4.0,15.0,NaN,NaN


In [31]:
# Replace missing data with 0.
train_trips = train_trips.fillna( 0 )
train_trips.head()

,trip_id,shopper_id,shopping_started_at,shopping_ended_at,shopping_time,model_1,store_id_105,store_id_115,store_id_123,store_id_126,...,Pantry_count,Pantry_quantity_sum,Frozen_count,Frozen_quantity_sum,Deli_count,Deli_quantity_sum,Bakery_count,Bakery_quantity_sum,Meat_and_Seafood_count,Meat_and_Seafood_quantity_sum
0,3119519,shopper_id_48539,2015-09-01 07:03:56,2015-09-01 07:30:56,1620.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,3.0,4.0,7.0,2.0,2.0,1.0,1.0
1,3119513,shopper_id_3775,2015-09-01 07:04:33,2015-09-01 07:40:33,2160.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3119516,shopper_id_4362,2015-09-01 07:23:21,2015-09-01 07:41:21,1080.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3119792,shopper_id_47659,2015-09-01 07:29:52,2015-09-01 08:55:52,5160.0,1.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0
4,3119922,shopper_id_11475,2015-09-01 07:32:21,2015-09-01 09:01:21,5340.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,8.0,4.0,15.0,0.0,0.0


In [32]:
target = train_trips[ "shopping_time" ]
data_train, data_val, target_train, target_val = train_test_split( train_trips, target, test_size=0.33, random_state=50 )

In [33]:
data_train.to_csv('data_train.csv', sep=',')
data_train_r = ro.r( "Dev = read.csv( 'data_train.csv', header = TRUE, sep = ',' )" )


In [35]:
print "Running the stepwise regression Model in R"
print ro.r( 'model = step( lm( shopping_time ~ model_1+store_id_105+store_id_115+store_id_123+store_id_126+store_id_148+store_id_29+store_id_3+store_id_31+store_id_5+store_id_54+store_id_6+store_id_78+store_id_90+item_count+quantity_sum+Popular_count+Popular_quantity_sum+Produce_count+Produce_quantity_sum+Dairy_and_Eggs_count+Dairy_and_Eggs_quantity_sum+Snacks_count+Snacks_quantity_sum+Beverages_count+Beverages_quantity_sum+Pantry_count+Pantry_quantity_sum+Frozen_count+Frozen_quantity_sum+Deli_count+Deli_quantity_sum+Bakery_count+Bakery_quantity_sum+Meat_and_Seafood_count+Meat_and_Seafood_quantity_sum, data = Dev ) )' )


Running the stepwise regression Model in R
Start:  AIC=1135714
shopping_time ~ model_1 + store_id_105 + store_id_115 + store_id_123 + 
    store_id_126 + store_id_148 + store_id_29 + store_id_3 + 
    store_id_31 + store_id_5 + store_id_54 + store_id_6 + store_id_78 + 
    store_id_90 + item_count + quantity_sum + Popular_count + 
    Popular_quantity_sum + Produce_count + Produce_quantity_sum + 
    Dairy_and_Eggs_count + Dairy_and_Eggs_quantity_sum + Snacks_count + 
    Snacks_quantity_sum + Beverages_count + Beverages_quantity_sum + 
    Pantry_count + Pantry_quantity_sum + Frozen_count + Frozen_quantity_sum + 
    Deli_count + Deli_quantity_sum + Bakery_count + Bakery_quantity_sum + 
    Meat_and_Seafood_count + Meat_and_Seafood_quantity_sum


Step:  AIC=1135714
shopping_time ~ model_1 + store_id_105 + store_id_115 + store_id_123 + 
    store_id_126 + store_id_148 + store_id_29 + store_id_3 + 
    store_id_31 + store_id_5 + store_id_54 + store_id_6 + store_id_78 + 
    store_id_90 

In [36]:
# Load the predictions for training set into a panda DataFrame
train_predict = list( ro.r( "response = predict( model, newdata = Dev ) " ) )

target_train = np.array( target_train )

# Print RMS Error from training
rms = np.sqrt( mean_squared_error(  target_train , train_predict ) )
print( rms )

1393.90446407
